# NILTMTK Class

NILMTK classes provide a means of bundling data and functioality together.  Creating a new class creates a new *type* of object, allowing new instances of that type to be made.   There are **MeterGroup**, **ElecMeter**, **Dataset** and other classes.  The ***Electric*** class is a common implementations of methods shared by ElectMeter and MeterGroup.   More detail UML (Unified Modeling Language) diagram can find [here](https://raw.githubusercontent.com/nilmtk/writing/master/figures/NILMTK_UML.png).

- Created Date : 18/4/2022
- Updated Date : 15/5/2022

**References:**
- J. Kelly and W. Knottenbelt, “[The UK-DALE dataset, domestic appliance-level electricity demand and whole-house demand from five UK homes](http://www.nature.com/articles/sdata20157)” Scientific Data, vol. 2, no. 1, p. 150007, Dec. 2015, doi: 10.1038/sdata.2015.7.
- [See metadata : dict](http://nilm-metadata.readthedocs.org/en/latest/dataset_metadata.html#elecmeter)
- [See meter_devices : dict, static class attribute](http://nilm-metadata.readthedocs.org/en/latest/dataset_metadata.html#meterdevice)
- [NILMTK's API Documentation](http://nilmtk.github.io/nilmtk/master/index.html)
- [NILMTK Documentation](https://github.com/nilmtk/nilmtk/tree/master/docs/manual)

# Initialization for Python and NILMTK

Let's kick-off to process and analysis the data with Python.

In [ ]:
import dateutil
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime

import nilmtk as ntk
import util as ut

## Define constant and global variable

In [ ]:
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = [15, 10]

RAW_FILENAME = "../Dataset/ukdale.h5"

START_TS ='2013-08-01 00:00:00'
END_TS='2013-08-31 23:59:59'

HOUSE_NUMBER = 1

# NILMTK - Class Objects

These objects consist of **Dataset**, **MetaGroup**, **MeterGroup** and **ElecMeter**.

## Create Dataset object 

This section initializes and create a Dataset object from nilmtk.dataset.DataSet

In [ ]:
# Create Dataset object for UK-DALE
ukdale_ds = ntk.DataSet(RAW_FILENAME)

# Set the duration window from START_TS to END_TS
ukdale_ds.set_window(start=START_TS,end=END_TS)

In [ ]:
type(ukdale_ds)

In [ ]:
# Return MetaGroup information for all buildings

ukdale_ds.elecs()

#
# Following APIs do not work
#
#ukdale_ds.describe()
#ukdale_ds.plot_mains_power_histograms()

## Create MetaGroup object for House Data 

This section initializes and create a House Data MetaGroup object from nilmtk.metergroup.MeterGroup

In [ ]:
# Create object for house_data and
# using global variable "HOUSE_NUMBER"
#
house_data = ukdale_ds.buildings[HOUSE_NUMBER].elec

type(house_data)

### Access MetaGroup for the brief high-level information

In [ ]:
print("Raw File at {}\n".format(RAW_FILENAME))

# Calling APIs function from custom python file
# Refer to "util.py" for the custom utility python function file
print("Select Timeframe")
ut.print_info_duration(START_TS, END_TS, "%Y-%m-%d %H:%M:%S")

print("\nThere are {} house/buildling with total of appliances.\n".format(len(ukdale_ds.buildings)))
for item in ukdale_ds.buildings:
    print("House {}, total appliances = {}.".format(item, len(ukdale_ds.buildings[item].elec.appliances)))
    print("Total Energy returns in 'kWh'")
    
    # Call utilility function from util.py
    ut.print_total_energy(ukdale_ds.buildings[item].elec.mains().total_energy())
    
    print("\n")

### Show house_data 1 of MeterGroup object

The collection of MeterGroup consists two main categories, **'appliances'** vs **'site_meter'**.  The **instance 54** for **house 1** consist various of appliances in the **'site_meter'**.


```
MeterGroup(meters=
...
     ElecMeter(instance=54, building=1, dataset='UK-DALE', site_meter, 
         appliances=[
           Appliance(type='immersion heater', instance=1), 
           Appliance(type='water pump', instance=1), 
           Appliance(type='security alarm', instance=1), 
           Appliance(type='fan', instance=2), 
           Appliance(type='drill', instance=1), 
           Appliance(type='laptop computer', instance=2)
         ]
     )
)```

In [ ]:
house_data

In [ ]:
#
# Functions can be found at metergroup.py
#

#print(house_data.pairwise_correlation())
print(house_data.is_site_meter())
print(house_data.sample_period())
print(house_data.dataset())

#
# Property of identifier
#
print(type(house_data.identifier))

print(type(house_data.instance()))
print(type(house_data.building()))
print(type(house_data.contains_meters_from_multiple_buildings()))

#
# Property of appliances
#
print(type(house_data.appliances))
print(type(house_data.dominant_appliances()))

# Specifies the key, which in string, like 'type' or 'categories' or 'room'
print(type(house_data.values_for_appliance_metadata_key('type')))

# Specifies MeterID in list for 3-tuples format
print(type(house_data.get_labels([(5, 1, 'UK-DALE')])))

## Work on Site Meter object for Mains

This section initializes and create a mains ElecMeter object from nilmtk.elecmeter.ElecMeter

In [ ]:
# Create a 'mains' variable
mains = house_data.mains()

print("Data Type of submeters is {}.".format(type(mains)))
print("Properties submeters of available_ac_types is {}.\n".format(mains.available_ac_types('power')))

mains

### Properties of ElecMeter Class

In [ ]:
print("Property of key is {}".format(mains.key))
print("Property of name is {}".format(mains.name))
print("Property of device is {}\n".format(mains.device))

### Functions of ElecMeter Class

In [ ]:
print("Function of get_timeframe() is {}".format(mains.get_timeframe()))
print("Function of sample_period() is {}".format(mains.sample_period()))
print("Function of is_site_meter() is {}".format(mains.is_site_meter()))

In [ ]:
# Tries to find the most dominant appliance on this meter,
# and then returns that appliance object.  Will return None
# if there are no appliances on this meter.

print("Function of dominant_appliance() is {}".format(mains.dominant_appliance()))

In [ ]:
# Returns a string describing this meter.

mains.label()

In [ ]:
#Finds available alternating current types for a specific physical quantity.

mains.available_ac_types('power')

In [ ]:
mains.available_physical_quantities()

In [ ]:
mains.available_columns()

In [ ]:
mains.total_energy()

In [ ]:
# Note: Suspect this API not working correctly or data issue.
mains.dropout_rate()

In [ ]:
print("Data Type of mains.good_sections() is {}".format(type(mains.good_sections())))

mains.good_sections()

### Load data from "mains" and return dataframe from the DataStore

In [ ]:
mains_df = next(mains.load(sample_period=6))

print("Data Type of mains_df is {}.  Total rows = {}".format(type(mains_df),len(mains_df)))

In [ ]:
mains_df.describe()

In [ ]:
print("**Head**\n{}\n\n**Tail**\n{}".format(mains_df.head(), mains_df.tail()))

In [ ]:
mains_df.plot()

In [ ]:
mains_in_min_df = mains_df["2013-08-01"].resample('1min').sum() * 10

print(mains_in_min_df.shape)

mains_in_min_df.describe()

In [ ]:
mains_in_min_df.head(24)

In [ ]:
mains_in_min_df.plot()

In [ ]:
mains_in_hour_df = mains_df["2013-08-01"].resample('60min').sum() * 10
mains_in_hour_df.head(24)

In [ ]:
mytitle = "Mains for a day at 2013-08-01"

ax = mains_in_hour_df.plot(title=mytitle,
                        xticks=mains_in_hour_df.index, 
                        grid=True)

# Settle x-axes label 
mains_in_hour_df["TS"] = mains_in_hour_df.index.strftime('%H:%M:%S')
ax.set_xticklabels(mains_in_hour_df.TS, rotation=45)

ax.set_xlabel('Time in Hourly')
ax.set_ylabel('Total Watt')

## Work on "Submeters"

This section initializes and create a Submeter ElecMeter object from nilmtk.metergroup.MeterGroup

In [ ]:
# Create a 'submeter' variable return MetaGroup object
submeters = house_data.submeters()

print("Data Type of submeters is {}.".format(type(submeters)))
print("Properties submeters of available_ac_types is {}.\n".format(submeters.available_ac_types('power')))

### Show the MetaGroup's of submeters content

In [ ]:
submeters

### Select an appliance from ElecMeter in MetaGroup - Washer Dryer 

In [ ]:
submeter_washer_dryer = house_data.select_using_appliances(type='washer dryer')

print(type(submeter_washer_dryer))
submeter_washer_dryer

In [ ]:
submeter_washer_dryer_df = submeter_washer_dryer.dataframe_of_meters()
submeter_washer_dryer_df.describe()

In [ ]:
df_wd = next(house_data['washer dryer'].load())
df_wd.describe()

### Select Top (more than one) Meters form submeters in MeterGroup

Only select the top K meters, according to energy.  Functions on the entire MeterGroup.  So if you mean to select the top K from only the submeters.  Default is k is 5.

In [ ]:
# Select top 5 objects from MeterGroup, return MeterGroup object 
mg_top = house_data.submeters().select_top_k(k=5)

print("\n\nData Type of mg_top is {}.".format(type(mg_top))) 

# Show the selected MeterGroup values
mg_top

In [ ]:
# Load data to dataframe from MeterGroup
mg_top_df = mg_top.dataframe_of_meters()

print("Is there any null value in dataframe = {}.\n".format(mg_top_df.isnull().values.any()))

# Change readable column name
mg_top_df.columns = house_data.get_labels(mg_top_df.columns)

# Show dataframe info for MeterGroup
mg_top_df.describe()

In [ ]:
mg_top_df.plot()

### Access an ElecMeter Object From MeterGroup object

In [ ]:
mgitem = mg_top.__getitem__(12)

print("Data Type of mgitem {}.".format(type(mgitem)))

print(mgitem.appliances)

print(mgitem.appliances[0])

print("\n")
print("Data Type of mgitem.appliances[0] {}.".format(type(mgitem.appliances[0])))
b = mgitem.appliances[0]
print(b.identifier)
print(b.n_meters)
print("\n")

print("\n")
print("Data Type of b.metadata {}.".format(type(b.metadata)))
print(b.metadata)
print(b.metadata.get("type"))
print(b.metadata.get("instance"))
print("\n")

print(b.categories)

# Comparing Meters Data

In [ ]:
df_ff = next(house_data['fridge freezer'].load())
df_ff.plot()

In [ ]:
df_ff.describe()

In [ ]:
mytitle = "The total of Fridge Freezer Power Usage from " + START_TS + " to " + END_TS
myx_label = "Day"
myy_label = "Active Power in Watt"

df_ff[('power', 'active')].resample('D').sum().plot(title=mytitle)
plt.xlabel(myx_label)
plt.ylabel(myy_label)
plt.show()

In [ ]:
sr_daily_ff = df_ff[('power', 'active')].resample('D').sum()

sr_daily_ff.index = sr_daily_ff.index.strftime('%Y-%m-%d').tolist()

sr_daily_ff

## Analysis of Main Vs Dish Washer for Daily

In [ ]:
# Combine two series for the daily of main and washing machine to a dataframe

# Get series of washing machine
df_ke = next(house_data['kettle'].load())
sr_of_ke = df_ke[('power', 'active')].resample('D').sum()

# Get series of main
df_of_mains = next(mains.load(sample_period=6))
sr_of_mains = df_of_mains[('power', 'apparent')].resample('D').sum()

# Merging two series into a dataframe
mains_ke_df=pd.concat([sr_of_mains,sr_of_ke],axis=1)

# Changing Columns Name and flatten MultiIndex Columns
mains_ke_df.columns = ["_".join(pair) for pair in mains_ke_df.columns]
mains_ke_df.columns = ['main_power_apparent', 'kettle_power_active']

# Generate Chart

mytitle = "The total of Power Usage - Mains vs. Kettle from " + START_TS + " to " + END_TS
myx_label = "Day"
myy_label = "Active Power in Watt"

ax = mains_ke_df.plot(title=mytitle)
ax.set_xticks(mains_ke_df.index)
mains_ke_df["TS"] = mains_ke_df.index.strftime('%y-%m-%d')
ax.set_xticklabels(mains_ke_df.TS, rotation=45)

plt.show()

In [ ]:
mains_ke_df

## Analysis of Main Vs Kettle for Hourly

In [ ]:
thedate = "2013-08-01"
mytitle = "Kettle for the day - " + thedate 

df_ke = next(house_data['kettle'].load())
df_of_mains = next(mains.load(sample_period=6))

sr_ke_daily = df_ke[('power', 'active')].resample('H').sum()
sr_mains_daily = df_of_mains[('power', 'apparent')].resample('H').sum()

sr_mains_theday = sr_mains_daily[thedate]
sr_ke_theday = sr_ke_daily[thedate]

print("Max value of mains  = {} at {}".format(sr_mains_theday.max(),
                                              sr_mains_theday.idxmax()))
print("Max value of kettle = {} at {}".format(sr_ke_theday.max(),
                                              sr_ke_theday.idxmax()))

# Merging two series into a dataframe
df_mains_ke_hourly = pd.concat([sr_mains_theday,sr_ke_theday],axis=1)

# Changing Columns Name and flatten MultiIndex Columns
df_mains_ke_hourly.columns = ["_".join(pair) for pair in df_mains_ke_hourly.columns]
df_mains_ke_hourly.columns = ['main_power_apparent', 'kettle_power_active']
df_mains_ke_hourly["TS"] = df_mains_ke_hourly.index.strftime('%H:%M:%S')

ax = df_mains_ke_hourly.plot(kind='line', title=mytitle)
ax.set_xticks(df_mains_ke_hourly.index)
ax.set_xticklabels(df_mains_ke_hourly.TS)

plt.show()

In [ ]:
df_mains_ke_hourly

# Workout & Findings

After the completion this notebook, prepare a simple presentation slide to express your summary and new discovery information from here.

- May able to further analysis and compare the top 5 appliances
- May perform highlevel grouping monthly, daily and hourly to identify appliances usage pattern.  This may help to further download the high frequency raw data